# ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# list of junk dirs 
junk_dirs = ['.ipynb_checkpoints']

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # ignore junk_dirs from path 
    for junk_dir in junk_dirs:
        if junk_dir in dirs:
            dirs.remove(junk_dir)
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*.csv'))
    

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)


# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','songTitle','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## Setting up connection to server

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine (127.0.0.1)
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    with REPLICATION = 
    {'class': 'SimpleStrategy', 'replication_factor': 1}"""
    )
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

# Modelling database tables based on given questions/queries.

## Question 1
### Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


#### turn question into query

In [8]:
query_select = "SELECT artist, songTitle, length from artist_song_playlist \
                WHERE sessionId = 338 and itemInSession = 4"

#### create table based on given query

In [9]:
query_create = "CREATE TABLE IF NOT EXISTS artist_song_playlist"
query_create = query_create + "(sessionId int, itemInSession int, artist text, songTitle text, length float, \
                                PRIMARY KEY (sessionId, itemInSession))"

try:
    session.execute(query_create)
except Exception as e:
    print(e)

#### Insert data from csv file into table

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    ## Assign the INSERT statements into the `query_insert` variable
    query_insert = """INSERT INTO artist_song_playlist (sessionId, itemInSession, artist, songTitle, length)"""
    query_insert = query_insert + " VALUES (%s, %s, %s, %s, %s)"
    for line in csvreader:
        # Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query_insert, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### run select_query to verify that the data have been inserted correctlly into the table

In [12]:
## Add in the SELECT statement to verify the data was entered into the table
try:
    rows = session.execute(query_select)
except Exception as e:
    print(e)

    
try:
    # use pretty table to display data in tabular form and include headings
    t = PrettyTable(['Artist', 'SongTitle', 'Length'])
    for row in rows:
        t.add_row([row.artist, row.songtitle, row.length])
    print(t)
except Exception as e:
    print(e) 

+-----------+---------------------------------+--------------------+
|   Artist  |            SongTitle            |       Length       |
+-----------+---------------------------------+--------------------+
| Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+-----------+---------------------------------+--------------------+


## Question 2
### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### turn question into query

In [13]:
query_select = "SELECT itemInSession, artist, songTitle, firstName, lastName from song_playlist_session \
                WHERE userId = 10 and sessionId = 182"

#### create table based on given query

In [14]:
query_create = "CREATE TABLE IF NOT EXISTS song_playlist_session "
query_create = query_create + "(userId int, sessionId int, itemInSession int, artist text, songTitle text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query_create)
except Exception as e:
    print(e)
                    

#### Insert data from csv file into table

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    ## Assign the INSERT statements into the `query_insert` variable
    query_insert = """INSERT INTO song_playlist_session (userId, sessionId, itemInSession, artist, songTitle, firstName, lastName)"""
    query_insert = query_insert + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
    for line in csvreader:
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query_insert, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### run select_query to verify that the data have been inserted correctlly into the table

In [16]:
## Add in the SELECT statement to verify the data was entered into the table
try:
    rows = session.execute(query_select)
except Exception as e:
    print(e)

    
try:
    # use pretty table to display data in tabular form and include headings
    t = PrettyTable(['ItemInSession', 'Artist', 'SongTitle', 'FirstName', 'LastName'])
    for row in rows:
        t.add_row([row.iteminsession, row.artist, row.songtitle, row.firstname, row.lastname])
    print(t)
except Exception as e:
    print(e) 

+---------------+-------------------+------------------------------------------------------+-----------+----------+
| ItemInSession |       Artist      |                      SongTitle                       | FirstName | LastName |
+---------------+-------------------+------------------------------------------------------+-----------+----------+
|       0       |  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie  |   Cruz   |
|       1       |    Three Drives   |                     Greece 2000                      |   Sylvie  |   Cruz   |
|       2       | Sebastien Tellier |                      Kilometer                       |   Sylvie  |   Cruz   |
|       3       |   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie  |   Cruz   |
+---------------+-------------------+------------------------------------------------------+-----------+----------+


## Question 3
### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### turn question into query

In [17]:
query_select = "SELECT firstName, lastName from user_song_playlist \
                WHERE songtitle = 'All Hands Against His Own'"

#### create table based on given query

In [18]:
query_create = "CREATE TABLE IF NOT EXISTS user_song_playlist "
query_create = query_create + "(userId int, songTitle text, firstName text, lastName text, PRIMARY KEY (songTitle, userId))"
try:
    session.execute(query_create)
except Exception as e:
    print(e)
                    

#### Insert data from csv file into table

In [19]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    ## Assign the INSERT statements into the `query_insert` variable
    query_insert = """INSERT INTO user_song_playlist (userId, songTitle, firstName, lastName)"""
    query_insert = query_insert + " VALUES (%s, %s, %s, %s)"
    for line in csvreader:
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query_insert, (int(line[10]), line[9], line[1], line[4]))

#### run select_query to verify that the data have been inserted correctlly into the table

In [20]:
try:
    rows = session.execute(query_select)
except Exception as e:
    print(e)    

try:
    # use pretty table to display data in tabular form and include headings
    t = PrettyTable(['FirstName', 'LastName'])
    for row in rows:
        t.add_row([row.firstname, row.lastname])
    print(t)
except Exception as e:
    print(e) 

+------------+----------+
| FirstName  | LastName |
+------------+----------+
| Jacqueline |  Lynch   |
|   Tegan    |  Levine  |
|    Sara    | Johnson  |
+------------+----------+


## Drop the tables before closing out the sessions

In [21]:
try:
    session.execute("drop table artist_song_playlist")
except Exception as e:
    print(e)
try:
    session.execute("drop table song_playlist_session")
except Exception as e:
    print(e)
try:    
    session.execute("drop table user_song_playlist")
except Exception as e:
    print(e)

## Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()